In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
target_train = df_train['y'].values
id_test = df_test['id'].values



In [3]:
#dummies = ["var9", "var18", "var22", "var23", "var24", "var25", "var27", "var29", "var30", "var31", "var33", "var39", "var41", "var44", "var47", "var48", "var49", "var50", "var51", "var53", "var54"]
#
#for col in dummies: # 100
#    df_train = pd.get_dummies(df_train, columns=[col])
#    df_test = pd.get_dummies(df_test, columns=[col])
#for col in df_test.columns:
#    
#    if col not in df_train.columns:
#        df_train[col] = 0
df_train["var22"] = df_train["var22"].astype(str)
df_train["var23"] = df_train["var23"].astype(str)
df_train["var24"] = df_train["var24"].astype(str)
df_train["histProd"] = df_train["var22"] + "-" +df_train["var23"] + "-" +df_train["var24"] + "-" 
df_test["var22"] = df_test["var22"].astype(str)
df_test["var23"] = df_test["var23"].astype(str)
df_test["var24"] = df_test["var24"].astype(str)
df_test["histProd"] = df_test["var22"] + "-" +df_test["var23"] + "-" +df_test["var24"] + "-" 


In [4]:
df_train["histProd"], _ = pd.factorize(df_train["histProd"])
df_test["histProd"], _ = pd.factorize(df_test["histProd"])

In [5]:
dummies = 20

for col in df_test.columns: # 100
    if (len(df_test[col].unique()) < dummies) and (len(df_test[col].unique()) == len(df_train[col].unique())):
        df_train = pd.get_dummies(df_train, columns=[col])
        df_test = pd.get_dummies(df_test, columns=[col])

In [6]:
train = np.array(df_train.drop(['y','id'], axis = 1))
test = np.array(df_test.drop(['id'], axis = 1))

xgb_preds = []

In [7]:
K = 5
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

In [8]:
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) # works for both type(y) == <class 'numpy.ndarray'> and type(y) == <class 'pandas.core.series.Series'>
    return 'f1',f1_score(t,y_bin)
err = 0
err2 = 0
err3 = 0
err4 = 0
err5 = 0
errLL = 0
cutPoint = 0.34
for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}
    #xgb_params = {'eta': 0.11, 'max_depth': 5, 'subsample': 0.4, 'colsample_bytree': 0.4, 'objective': 'binary:logistic', 'min_child_weight': 15, 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, verbose_eval=50, early_stopping_rounds=100)
                        
    xgb_pred = model.predict(d_test)
    pG = model.predict(d_valid)

    errLL += log_loss(valid_y, pG)
    p = pG
    p[p > cutPoint] = 1
    p[p != 1] = 0
    err += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint+0.01)] = 1
    p[p != 1] = 0
    err2 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint+0.02)] = 1
    p[p != 1] = 0
    err3 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint-0.02)] = 1
    p[p != 1] = 0
    err4 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint-0.01)] = 1
    p[p != 1] = 0
    err5 += f1_score(valid_y, p)
    print(err)
    print(model.get_score(importance_type='gain'))
    xgb_preds.append(list(xgb_pred))

[17:58:51] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.84887	valid-auc:0.82489
[50]	train-auc:0.87772	valid-auc:0.85538
[100]	train-auc:0.88556	valid-auc:0.86293
[150]	train-auc:0.89544	valid-auc:0.87021
[200]	train-auc:0.90606	valid-auc:0.87672
[250]	train-auc:0.91451	valid-auc:0.88031
[300]	train-auc:0.92182	valid-auc:0.88290
[350]	train-auc:0.92763	valid-auc:0.88538
[400]	train-auc:0.93290	valid-auc:0.88678
[450]	train-auc:0.93749	valid-auc:0.88750
[500]	train-auc:0.94137	valid-auc:0.88818
[550]	train-auc:0.94536	valid-auc:0.88869
[600]	train-auc:0.94897	valid-auc:0.88878
[650]	train-auc:0.95197	valid-auc:0.88941
[700]	train-auc:0.95488	valid-auc:0.88993
[750]	train-auc:0.95768	valid-auc:0.89051
[80

In [9]:
df_test["histProd"]

0         0
1         1
2         2
3         0
4         3
         ..
21178     0
21179    17
21180     5
21181     0
21182     0
Name: histProd, Length: 21183, dtype: int64

In [10]:
print(err/5) #print(err/4) #6711266
print(err2/5) #print(err/4) #6711266
print(err3/5) #print(err/4) #6711266
print(err4/5) #print(err/4) #6711266
print(err5/5) #print(err/4) #6711266
print(errLL/5) #print(err/4) #6711266

0.6717580288914998
0.6717580288914998
0.6717580288914998
0.6717580288914998
0.6717580288914998
0.30731110454339106


In [11]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

output = pd.DataFrame({'id': id_test, 'predicted': preds})
output['predicted'][output['predicted'] > cutPoint] = 1
output['predicted'][output['predicted'] != 1] = 0
output['predicted'] = output['predicted'].astype(int)
output.to_csv("../data/output/{}-foldCV_avg_sub_dummy_hist_cutpoint{}_error{}_logloss{}.csv".format(K,cutPoint, err/5, errLL/5), index=False)   

<ipython-input-11-42f59777eed3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] > cutPoint] = 1
<ipython-input-11-42f59777eed3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] != 1] = 0


In [99]:
sub = pd.read_csv("./5-foldCV_avg_sub_36_dummy.csv")

In [100]:
sub["v"] = output["predicted"]

In [101]:
sub["v2"] = sub["v"] - sub["predicted"]

In [102]:
sub[sub["v2"] != 0]

,id,predicted,v,v2


In [84]:
119

152